In [1]:
# https://msprime.readthedocs.io/en/stable/tutorial.html#demography

In [56]:
from math import (exp, log)
import numpy as np
import msprime

In [85]:
RHO_HUMAN = 1.6*10e-9
MU_HUMAN = 1.25*10e-8
RHO_LIMIT = (log(RHO_HUMAN)-100, log(RHO_HUMAN)+100)
MU_LIMIT = (log(MU_HUMAN)-100, log(MU_HUMAN)+100)

LENGTH_NORMALIZE_CONST = 4
ZIPPED = False
NUMBER_OF_EVENTS_LIMITS = (1, 20)
MAX_T_LIMITS = (0.01, 30)
LAMBDA_EXP = 1.0
POPULATION_LIMITS = (250, 100000)
POPULATION = 5000

IS_SEMMETRIC = True

N = 20

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

In [18]:
population_configurations = [
    msprime.PopulationConfiguration(
        sample_size=1, initial_size=100),
    msprime.PopulationConfiguration(
        sample_size=1, initial_size=200, growth_rate=-0.01)
]
# What's wrong with growth_rate? Why population does growth only if it's negative?

In [39]:
def create_N_population(populations_parametrs: dict) -> list:
    """
    population_parametrs: {population_name:int : [sample_size:int, initial_size:float, growth_rate:float]}
    """
    populations_configuration = []
    for population_name, parametrs in populations_parametrs.items():
        if len(parametrs)  == 3:
            sample_size, initial_size, growth_rate = parametrs
        elif len(parametrs)  == 2:
            sample_size, initial_size = parametrs
        else:
            raise f"Number ov parametrs error: expected 2 or 3, but got {len(parametrs)}"
        populations_configuration.append(
            msprime.PopulationConfiguration(
                sample_size=sample_size, 
                initial_size=initial_size, 
                growth_rate=growth_rate)
        )
    return populations_configuration

In [51]:
def create_random_migration_matrix(N: int, max_migration_rate, is_semmetric = IS_SEMMETRIC):
    migration_matrix = np.zeros(shape=(N,N))
    for i in range(N):
        for j in range(i + 1,N):
            migration_matrix[i][j] = np.random.random()*max_migration_rate
            if is_semmetric:
                migration_matrix[j][i] = migration_matrix[i][j]
            else:
                migration_matrix[j][i] = np.random.random()*max_migration_rate
    return migration_matrix

    demographic_events = [
        # CEU and CHB merge into B with rate changes at T_EU_AS
        
        msprime.MassMigration(
         time=T_EU_AS, source=2, destination=1, proportion=1.0)
            ,
        msprime.MigrationRateChange(time=T_EU_AS, rate=0),
        msprime.MigrationRateChange(
            time=T_EU_AS, rate=m_AF_B, matrix_index=(0, 1)),
        msprime.MigrationRateChange(
            time=T_EU_AS, rate=m_AF_B, matrix_index=(1, 0)),
        msprime.PopulationParametersChange(
            time=T_EU_AS, initial_size=N_B, growth_rate=0, population_id=1),
        # Population B merges into YRI at T_B
        msprime.MassMigration(
            time=T_B, source=1, destination=0, proportion=1.0),
        msprime.MigrationRateChange(time=T_B, rate=0),
        # Size changes to N_A at T_AF
        msprime.PopulationParametersChange(
            time=T_AF, initial_size=N_A, population_id=0)
    ]

In [119]:
MAX_MIGRATION_RATE = 10

def create_MassMigration(time: float, number_of_populations: int, populations: list) -> list:
    if number_of_populations == 1:
        raise "Only one population"
    source, destination = 0,0
    while source == destination:
        source, destination = np.random.randint(number_of_populations,size=2)
    proportion = min(np.random.rand(), 1.0) # learn max of np.random.rand()
    return [msprime.MassMigration(time=time,source=source,destination=destination, proportion=proportion)]

def create_MigrationRateChange(time: float, number_of_populations: int, populations: list) -> list:
    rate = np.random.random()*MAX_MIGRATION_RATE
    i,j = np.random.randint(low=number_of_populations,size=(2))
    while i == j:
        i,j = np.random.randint(low=number_of_populations,size=(2))
    migration = [msprime.MigrationRateChange(time=time,rate=rate,matrix_index=(i,j))]
    if IS_SEMMETRIC:
        migration.append(msprime.MigrationRateChange(time=time,rate=rate,matrix_index=(j,i)))
    return migration
    
def create_PopulationParametersChange(time: float, number_of_populations: int, populations: list) -> list:
    population_id = np.random.randint(number_of_populations)
    size = int(np.random.beta(a=2, b=5) * populations[population_id])
    while size <= 0:
        size = int(np.random.beta(a=2, b=5) * populations[population_id])
    return [msprime.PopulationParametersChange(time=time, initial_size=size, population_id=population_id)]

events = {
    "MassMigration": create_MassMigration,
    "MigrationRateChange": create_MigrationRateChange,
    "PopulationParametersChange": create_PopulationParametersChange,
}

events = [create_MassMigration,create_MigrationRateChange,create_PopulationParametersChange]

def create_demographic_events(number_of_populations: int, populations: list) -> list:
    """
    """
    low, high = NUMBER_OF_EVENTS_LIMITS
    number_of_events = np.random.randint(low=low, high=high)
    times = sorted(np.random.exponential(LAMBDA_EXP, size=number_of_events))
    low, high = MAX_T_LIMITS
    max_t = np.random.uniform(low=low, high=high)
    
    alpha = 1.0
    beta = np.log(max_t + 1)/times[-1]

    def to_exp_time(time: float) -> float:
        # time -> exponentional time
        return alpha*(np.exp(beta*time) - 1)
    
    exp_times = [to_exp_time(t) for t in times]
    
    demographic_events = []
    for time in exp_times:
        event = np.random.choice(events)(time, number_of_populations, populations)
        for ev in event:
            demographic_events.append(ev)
    return demographic_events

In [163]:
create_demographic_events(3,[10,10,10])

[{'type': 'mass_migration', 'time': 0.006401276473684492, 'source': 1, 'dest': 0, 'proportion': 0.8245140468968338},
 {'type': 'migration_rate_change', 'time': 0.10506235452228463, 'rate': 3.9692327620159604, 'matrix_index': (2, 1)},
 {'type': 'migration_rate_change', 'time': 0.10506235452228463, 'rate': 3.9692327620159604, 'matrix_index': (1, 2)},
 {'type': 'migration_rate_change', 'time': 0.19733186000222047, 'rate': 6.9319394082268095, 'matrix_index': (2, 1)},
 {'type': 'migration_rate_change', 'time': 0.19733186000222047, 'rate': 6.9319394082268095, 'matrix_index': (1, 2)},
 {'type': 'migration_rate_change', 'time': 0.29282503637899926, 'rate': 1.6748258225906976, 'matrix_index': (0, 2)},
 {'type': 'migration_rate_change', 'time': 0.29282503637899926, 'rate': 1.6748258225906976, 'matrix_index': (2, 0)},
 {'type': 'mass_migration', 'time': 0.5667491814691707, 'source': 1, 'dest': 0, 'proportion': 0.9362122462436898},
 {'type': 'population_parameters_change', 'time': 0.70403858879045

In [121]:
np.random.randint(2)

0

In [168]:
list(create_random_migration_matrix(3,1))

[array([0.        , 0.34124783, 0.38019562]),
 array([0.34124783, 0.        , 0.39882278]),
 array([0.38019562, 0.39882278, 0.        ])]

In [169]:
dd = msprime.DemographyDebugger(
    population_configurations=population_configurations,
    migration_matrix=create_random_migration_matrix(2,1),
    demographic_events=create_demographic_events(2,[10,10])
)
dd.print_history()

Model =  hudson(reference_size=1)
Epoch: 0 -- 0.062047872452412944 generations
     start     end      growth_rate |     0        1    
   -------- --------       -------- | -------- -------- 
0 |   100      100                0 |     0      0.58   
1 |   200      200            -0.01 |   0.58       0    

Events @ generation 0.062047872452412944
   - Mass migration: Lineages moved with probability 0.19455832318530097 backwards in time with source 0 & dest 1
                     (equivalent to migration from 1 to 0 forwards in time)
Epoch: 0.062047872452412944 -- 0.5041650462750045 generations
     start     end      growth_rate |     0        1    
   -------- --------       -------- | -------- -------- 
0 |   100      100                0 |     0      0.58   
1 |   200      201            -0.01 |   0.58       0    

Events @ generation 0.5041650462750045
   - Migration rate change for (1, 0) to 4.858717591863257
   - Migration rate change for (0, 1) to 4.858717591863257
Epoch: 0.5041

In [159]:
generator = msprime.simulate(
        #sample_size=2,
        recombination_rate=RHO_HUMAN,
        mutation_rate=0.1,
        random_seed=42,
        model="hudson",
        length=20,
        num_replicates=1,
        population_configurations=population_configurations,
        migration_matrix=create_random_migration_matrix(3,0.001),
        demographic_events=create_demographic_events(3,[10,10])
)

ValueError: migration matrix must be a N x N square matrix encoded as a list-of-lists, where N is the number of populations defined in the population_configurations. The diagonal elements of this matrix must be zero. For example, a valid matrix for a 3 population system is [[0, 1, 1], [1, 0, 1], [1, 1, 0]]

In [156]:
tr = []
for i in generator:
    tr.append(i)

In [157]:
for variant in tr[0].variants():
    print(
        variant.site.id, variant.site.position,
        variant.alleles, variant.genotypes, sep="\t")

0	0.03759766463190317	('0', '1')	[0 0 1 0]
1	0.10459224227815866	('0', '1')	[1 1 0 0]
2	0.18663234543055296	('0', '1')	[1 1 0 0]
3	0.2421696064993739	('0', '1')	[0 0 0 1]
4	0.4188992315903306	('0', '1')	[0 0 0 1]
5	0.4586562281474471	('0', '1')	[0 0 1 0]
6	0.5814275564625859	('0', '1')	[0 0 1 0]
7	0.7062270725145936	('0', '1')	[1 1 0 0]
8	0.7685270579531789	('0', '1')	[0 0 0 1]
9	0.833458062261343	('0', '1')	[0 0 0 1]
10	0.8390244701877236	('0', '1')	[0 0 0 1]
11	0.8634747378528118	('0', '1')	[1 1 0 0]
12	0.996840582229197	('0', '1')	[0 0 1 0]
13	1.0560792265459895	('0', '1')	[0 0 1 0]
14	1.1227221135050058	('0', '1')	[0 0 1 0]
15	1.1410944676026702	('0', '1')	[1 1 0 1]
16	1.2508445233106613	('0', '1')	[0 0 1 0]
17	1.333040869794786	('0', '1')	[0 0 0 1]
18	1.379160420037806	('0', '1')	[1 1 0 1]
19	1.425134646706283	('0', '1')	[0 0 0 1]
20	1.4416866563260555	('0', '1')	[0 0 0 1]
21	1.6434334171935916	('0', '1')	[0 0 0 1]
22	1.6958403214812279	('0', '1')	[0 0 0 1]
23	1.967912930995226	('

In [82]:
i,j = np.random.randint(low=2,size=(2))

In [83]:
i,j

(1, 0)

In [84]:
a = []
b = [1,2]
a.append(*b)

TypeError: append() takes exactly one argument (2 given)